In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
#import get to call a get request on the site
from requests import get
import re

## Webscraping each city

In [73]:
#replace each city unique link here (for example purposes this is Houston, TX)
url = 'https://Cincinnati.craigslist.org/d/motorcycles-scooters/search/mca'

### Creating all the links to parse through for each city

In [74]:
#get the first page of motorcyle listings
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
#find the total number of posts to find the limit of the pagination
results_num = html_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

#each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
pages = np.arange(0, results_total+1, 120)

In [75]:
#get the links for each pagination
links = []

for page in pages:
    #get request
    links.append(url 
                   + "?s=" #the parameter for defining the page number 
                   + str(page)) #the page number in the pages array from earlier


                         
links

['https://Cincinnati.craigslist.org/d/motorcycles-scooters/search/mca?s=0',
 'https://Cincinnati.craigslist.org/d/motorcycles-scooters/search/mca?s=120']

### Scrape each page to get the individual listings on the page with their price and unique link

In [76]:
price_list= []
url_list = []
title_list = []

for link in links:
    response = get(link)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    results = html_soup.find(class_='rows')
    results.prettify()
    bike_elements = results.find_all('li', class_ ='result-row')
    for bike_elem in bike_elements:
        
        #   print(bike_elem.text)
        price_elem =bike_elem.find('span', class_='result-price')
        try:
            url_elem =bike_elem.find('a', class_='result-image gallery')['href']
        except TypeError:
            pass
        title_elem =bike_elem.find('a', class_="result-title hdrlnk")
        title_list.append(title_elem.text.strip())
        url_list.append(url_elem)
        price_list.append(price_elem.text.strip())
    #   print(price_list)
        print(title_elem.text.strip())
        print(url_elem)
        print(price_elem.text.strip())
        print()

2017 Honda Grom
https://cincinnati.craigslist.org/mcy/d/fairborn-2017-honda-grom/7398561907.html
$2,900

Harley Davidson Street Bob
https://cincinnati.craigslist.org/mcy/d/cincinnati-harley-davidson-street-bob/7398490132.html
$9,000

harley davidson sportster 883
https://cincinnati.craigslist.org/mcy/d/loveland-harley-davidson-sportster-883/7398303370.html
$5,000

Honda Goldwing Trike
https://cincinnati.craigslist.org/mcy/d/antioch-honda-goldwing-trike/7398270169.html
$18,000

2020 Honda SXS10S2X Talon X
https://cincinnati.craigslist.org/mcd/d/hamilton-2020-honda-sxs10s2x-talon/7398231660.html
$20,750

2002 Suzuki rm125
https://cincinnati.craigslist.org/mcd/d/hamilton-2020-honda-sxs10s2x-talon/7398231660.html
$2,100

2013 Honda CBR 250R
https://cincinnati.craigslist.org/mcy/d/cincinnati-2013-honda-cbr-250r/7398112578.html
$3,200

2013 Yamaha WR450F-24 MILES
https://cincinnati.craigslist.org/mcy/d/hamilton-2013-yamaha-wr450f-24-miles/7398077455.html
$4,300

2008 Honda ST1300
https://cin

2018 Triumph Street Twin ABS Parallel Twin 900 cc
https://cincinnati.craigslist.org/mcd/d/cincinnati-2018-triumph-street-twin-abs/7390464554.html
$7,999

2019 Harley-Davidson FXDR 114
https://cincinnati.craigslist.org/mcd/d/kings-mills-2019-harley-davidson-fxdr/7390304999.html
$19,999

2000 Harley Davidson 883 Sportster
https://cincinnati.craigslist.org/mcy/d/seaman-2000-harley-davidson-883/7390293706.html
$2,800

2015 Honda CB500F
https://cincinnati.craigslist.org/mcy/d/seaman-2000-harley-davidson-883/7390293706.html
$2,500

Suzuki GSX F 750 Katana
https://cincinnati.craigslist.org/mcy/d/amelia-suzuki-gsx-750-katana/7390119305.html
$12,345

2021 yz250fx
https://cincinnati.craigslist.org/mcy/d/florence-2021-yz250fx/7389943121.html
$10,000

Honda CB750 K4
https://cincinnati.craigslist.org/mcy/d/south-lebanon-honda-cb750-k4/7389939126.html
$6,500

2020 Harley-Davidson Ultra Limited
https://cincinnati.craigslist.org/mcd/d/kings-mills-2020-harley-davidson-ultra/7389858953.html
$30,999

Ben

### With the individual links that were appended to the list, scrape each link for their attributes

In [77]:
bike_title_2= []
attribute_list =[]

for i, url in enumerate(url_list):    
    bike_url = url
    bike_page = requests.get(bike_url)
    bike_soup =BeautifulSoup(bike_page.content, 'html.parser')
    attributes = bike_soup.find_all('p', class_='attrgroup')
#     for attribute in attributes:
    if attributes:
        bike_title_2.append(attributes[0].text.strip())
        attribute_list.append((attributes[1].text.strip()))
    else:
        bike_title_2.append('none')
        attribute_list.append('none')
        print(i, url)

Making sure all the lists are equal length

In [78]:
print(len(bike_title_2))
print(len(attribute_list))
print(len(price_list))
print(len(title_list))
print(len(url_list))

190
190
190
190
190


### Concatanating them to a single DataFrame

In [79]:
df = pd.DataFrame(list(zip(price_list,url_list,title_list,bike_title_2, attribute_list, )),columns=['Price','URL', 'title', 'bikeTitle', 'attributes'])
df

,Price,URL,title,bikeTitle,attributes
0,"$2,900",https://cincinnati.craigslist.org/mcy/d/fairbo...,2017 Honda Grom,Honda Grom,condition: excellent\n\nfuel: gas\n\nodometer:...
1,"$9,000",https://cincinnati.craigslist.org/mcy/d/cincin...,Harley Davidson Street Bob,2012 harley davidson,condition: excellent\n\nfuel: gas\n\nodometer:...
2,"$5,000",https://cincinnati.craigslist.org/mcy/d/lovela...,harley davidson sportster 883,2007 harley-davidson sportster,condition: excellent\n\nengine displacement (C...
3,"$18,000",https://cincinnati.craigslist.org/mcy/d/antioc...,Honda Goldwing Trike,2013 Honda Goldwing,condition: excellent\n\nengine displacement (C...
4,"$20,750",https://cincinnati.craigslist.org/mcd/d/hamilt...,2020 Honda SXS10S2X Talon X,2020 HONDA SXS10S2X TALON X,condition: excellent\n\nengine displacement (C...
...,...,...,...,...,...
185,"$15,000",https://cincinnati.craigslist.org/mcy/d/cincin...,Harley Panhead- trade,Harley Davidson Panhead,fuel: gas\n\nodometer: 1000\n\ntransmission: m...
186,"$7,000",https://cincinnati.craigslist.org/mcy/d/cincin...,2012 Kawasaki Vulcan 900 Classic,2012 kawasaki vulcan 900 classic,condition: excellent\n\nengine displacement (C...
187,"$8,000",https://cincinnati.craigslist.org/mcy/d/hamilt...,2013 Triumph Tiger Explorer 1200 ABS,triumph tiger explorer abs,condition: excellent\n\nengine displacement (C...
188,"$1,500",https://cincinnati.craigslist.org/mcy/d/floren...,2006 Genuine Black Cat 🐈‍⬛ 50cc 2 stroke motor...,2006 Genuine Black Cat 50,condition: like new\n\nengine displacement (CC...


### Giving each attributes its own column

Attribute column cleaning to start

In [80]:
#Splitting attributes to make it readable
for row in range(len(df)):
    attributes = df['attributes'][row].split(', ')[1:]
    for attribute in attributes:
        sub_attributes = attribute.split(': ')
        try:
            df.at[row,sub_attributes[0]]= sub_attributes[1]
        except:
            pass
        
        

In [81]:
pd.set_option('display.max_columns',None)

In [82]:
for i in range(len(attribute_list)):
    df['attributes'][i] = df['attributes'][i].strip()
    df['attributes'][i] = df['attributes'][i].replace('\n', '')
    df['attributes'][i] = df['attributes'][i].replace('VIN', ', VIN')
    df['attributes'][i] = df['attributes'][i].replace('condition', ', condition')
    df['attributes'][i] = df['attributes'][i].replace('fuel', ', fuel')
    df['attributes'][i] = df['attributes'][i].replace('paint', ', paint')
    df['attributes'][i] = df['attributes'][i].replace('title', ', title')
    df['attributes'][i] = df['attributes'][i].replace('engine', ', engine')
    df['attributes'][i] = df['attributes'][i].replace('odometer', ', odometer')
    df['attributes'][i] = df['attributes'][i].replace('delivery', ', delivery')
    df['attributes'][i] = df['attributes'][i].replace('transmission', ', transmission')
    df['attributes'][i] = df['attributes'][i].replace('type', ', type')

Each attribute gets its own column

In [83]:
for row in range(len(df)):
    attributes = df['attributes'][row].split(', ')[1:]
    for attribute in attributes:
        sub_attributes = attribute.split(': ')
        try:
            df.at[row,sub_attributes[0]]= sub_attributes[1]
        except:
            pass
        

In [84]:
df

,Price,URL,title,bikeTitle,attributes,condition,fuel,odometer,title status,transmission,paint color,engine displacement (CC),type,VIN
0,"$2,900",https://cincinnati.craigslist.org/mcy/d/fairbo...,2017 Honda Grom,Honda Grom,", condition: excellent, fuel: gas, odometer: 1...",excellent,gas,1300,clean,manual,NaN,NaN,NaN,NaN
1,"$9,000",https://cincinnati.craigslist.org/mcy/d/cincin...,Harley Davidson Street Bob,2012 harley davidson,", condition: excellent, fuel: gas, odometer: 4...",excellent,gas,4800,clean,manual,black,NaN,NaN,NaN
2,"$5,000",https://cincinnati.craigslist.org/mcy/d/lovela...,harley davidson sportster 883,2007 harley-davidson sportster,", condition: excellent, engine displacement (C...",excellent,gas,16000,clean,manual,blue,883,cruiser,NaN
3,"$18,000",https://cincinnati.craigslist.org/mcy/d/antioc...,Honda Goldwing Trike,2013 Honda Goldwing,", condition: excellent, engine displacement (C...",excellent,gas,8800,clean,manual,black,1800,trike,NaN
4,"$20,750",https://cincinnati.craigslist.org/mcd/d/hamilt...,2020 Honda SXS10S2X Talon X,2020 HONDA SXS10S2X TALON X,", condition: excellent, engine displacement (C...",excellent,gas,211,clean,automatic,red,1000,other,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,"$15,000",https://cincinnati.craigslist.org/mcy/d/cincin...,Harley Panhead- trade,Harley Davidson Panhead,", fuel: gas, odometer: 1000, transmission: manual",NaN,gas,1000,NaN,manual,NaN,NaN,NaN,NaN
186,"$7,000",https://cincinnati.craigslist.org/mcy/d/cincin...,2012 Kawasaki Vulcan 900 Classic,2012 kawasaki vulcan 900 classic,", condition: excellent, engine displacement (C...",excellent,gas,17543,clean,manual,blackstreet legal,900,cruiser,NaN
187,"$8,000",https://cincinnati.craigslist.org/mcy/d/hamilt...,2013 Triumph Tiger Explorer 1200 ABS,triumph tiger explorer abs,", condition: excellent, engine displacement (C...",excellent,gas,12200street legal,NaN,manual,NaN,1200,NaN,NaN
188,"$1,500",https://cincinnati.craigslist.org/mcy/d/floren...,2006 Genuine Black Cat 🐈‍⬛ 50cc 2 stroke motor...,2006 Genuine Black Cat 50,", condition: like new, engine displacement (CC...",like new,gas,60street legal,clean,automatic,NaN,50,scooter,NaN


Getting year from the bikeTitle

In [85]:
#getting year from BikeTitle in case it doesnt have it in the title
for row in range(len(df)):
    df.at[row, 'years'] = df['bikeTitle'][row][:4]

Getting year from the title to compare

In [86]:
import re 

def regex_year(string):
    try:
        return re.match(r"[\d]+", string).group()
    except:
        return re.match(r"[\d]+", string)

In [87]:
df['year'] = list(map(regex_year, df['title']))


### Then pickle it to use the data frame to concat with the other cities

In [88]:
df.to_pickle('cincinnati_motorcycle')

In [89]:
df_pickle=pd.read_pickle('cincinnati_motorcycle')
df_pickle

,Price,URL,title,bikeTitle,attributes,condition,fuel,odometer,title status,transmission,paint color,engine displacement (CC),type,VIN,years,year
0,"$2,900",https://cincinnati.craigslist.org/mcy/d/fairbo...,2017 Honda Grom,Honda Grom,", condition: excellent, fuel: gas, odometer: 1...",excellent,gas,1300,clean,manual,NaN,NaN,NaN,NaN,Hond,2017
1,"$9,000",https://cincinnati.craigslist.org/mcy/d/cincin...,Harley Davidson Street Bob,2012 harley davidson,", condition: excellent, fuel: gas, odometer: 4...",excellent,gas,4800,clean,manual,black,NaN,NaN,NaN,2012,None
2,"$5,000",https://cincinnati.craigslist.org/mcy/d/lovela...,harley davidson sportster 883,2007 harley-davidson sportster,", condition: excellent, engine displacement (C...",excellent,gas,16000,clean,manual,blue,883,cruiser,NaN,2007,None
3,"$18,000",https://cincinnati.craigslist.org/mcy/d/antioc...,Honda Goldwing Trike,2013 Honda Goldwing,", condition: excellent, engine displacement (C...",excellent,gas,8800,clean,manual,black,1800,trike,NaN,2013,None
4,"$20,750",https://cincinnati.craigslist.org/mcd/d/hamilt...,2020 Honda SXS10S2X Talon X,2020 HONDA SXS10S2X TALON X,", condition: excellent, engine displacement (C...",excellent,gas,211,clean,automatic,red,1000,other,NaN,2020,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,"$15,000",https://cincinnati.craigslist.org/mcy/d/cincin...,Harley Panhead- trade,Harley Davidson Panhead,", fuel: gas, odometer: 1000, transmission: manual",NaN,gas,1000,NaN,manual,NaN,NaN,NaN,NaN,Harl,None
186,"$7,000",https://cincinnati.craigslist.org/mcy/d/cincin...,2012 Kawasaki Vulcan 900 Classic,2012 kawasaki vulcan 900 classic,", condition: excellent, engine displacement (C...",excellent,gas,17543,clean,manual,blackstreet legal,900,cruiser,NaN,2012,2012
187,"$8,000",https://cincinnati.craigslist.org/mcy/d/hamilt...,2013 Triumph Tiger Explorer 1200 ABS,triumph tiger explorer abs,", condition: excellent, engine displacement (C...",excellent,gas,12200street legal,NaN,manual,NaN,1200,NaN,NaN,triu,2013
188,"$1,500",https://cincinnati.craigslist.org/mcy/d/floren...,2006 Genuine Black Cat 🐈‍⬛ 50cc 2 stroke motor...,2006 Genuine Black Cat 50,", condition: like new, engine displacement (CC...",like new,gas,60street legal,clean,automatic,NaN,50,scooter,NaN,2006,2006
